<a href="https://colab.research.google.com/github/YuxinHuang01/WANNA-RUN.github.io/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import some necessary library

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [4]:
# Build the 18-layer ResNet model
class BasicBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(residual)
        out = F.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 , num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)

        return out


def resnet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


In [5]:
# Initialize two ResNet 18 models
device = "cuda" if torch.cuda.is_available() else "cpu"
model_SGD = resnet18().to(device)
model_ADAM = resnet18().to(device)

In [6]:
# Load CIFAR 10 Datasets

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:03<00:00, 48501676.12it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
# Train function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= num_batches
    print(f"Avg Train loss: {loss:>8f} \n")
    return train_loss

In [8]:
# Test function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss, correct

In [9]:
import torch.optim as optim

# Define the loss function
criterion = nn.CrossEntropyLoss()

# set the optimizer as SGD with Momentum
# Please finish this part
optimizer_SGD = optim.SGD(params = model_SGD.parameters(), lr = 0.001, momentum = 0.9)

In [9]:
# Train model_SGD by SGD with Momentum optimization algorithm
# Please add code to finish this part.
train(trainloader, model_SGD, criterion, optimizer_SGD)


Avg Train loss: 1.223069 



1.2824005761058073

In [11]:
# @title
# Visualize the results (You can refer to other tutorial notebooks)
import matplotlib.pyplot as plt

epochs = 30
for epoch in range(1, epochs + 1):
  train(trainloader, model_SGD, criterion, optimizer_SGD)
  #test(model, device, test_loader, visualizer)


KeyboardInterrupt: 

In [12]:
# @title
# Visualize the results (You can refer to other tutorial notebooks)
import matplotlib.pyplot as plt
import torch.utils.data as Data

## Add codes to finish this part and save the images. Add the images to your assignment solution.

# Firstly, we prepare dataloader to stochasticly sample mini-batch data from the whole dataset
Epochs=31
losses=[]

for i in range(Epochs):
  for step, (x_batch, y_batch) in enumerate(trainloader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        model_SGD.train()
        # forward：calculate loss
        prediction = model_SGD(x_batch)     # input x and predict based on x
        loss = criterion(prediction, y_batch)     # must be (1. nn output, 2. target)


        # backward： calculate the gradient
        optimizer_SGD.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        # Update parameters
        optimizer_SGD.step()        # apply gradients
        model_SGD.eval()
  losses.append(loss.item().cpu())
  if i % 5 == 0:
    print('Epoch:', i, 'Loss value:', loss.item())

# Plot the training loss
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.title('Training Loss over Epochs')
plt.savefig('training_loss.png')
plt.show()

AttributeError: 'float' object has no attribute 'cpu'

In [ ]:
  if i%5 == 0:
        # Drawing
        plt.clf()
        image = x_batch[0].cpu()  # Take the first image from the batch
        image = image / 2 + 0.5  # Unnormalize the image
        plt.imshow(image.permute(1, 2, 0).numpy())  # Display the image
        plt.title('Label: ' + classes[y_batch[0].int().item()])  # Display the corresponding label
        plt.savefig(f'figure_epoch_{i}.png')  # Save the figure
        plt.show()
        plt.pause(0.8)
        print('Loss value:', loss.item())

In [ ]:
# Repeat the training and testing procedure for model_ADAM
optimizer_ADAM = optim.Adam(params=model_ADAM.parameters(),betas=[0.9,0.99],lr=0.01)

# Visualize the results and save the images. Add the images to your assignment solution.
